In [16]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+1):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)


def Unpack_Histo(histogram_file):
    bin_content = []
    bin_content_uncer =[]
    for i in range(1,histogram_file.GetNbinsX()+1):
        bin_content.append(histogram_file.GetBinContent(i))
        bin_content_uncer.append(histogram_file.GetBinError(i))
        
    return np.array(bin_content),np.array(bin_content_uncer)

############# CONFIGURING THE JOB ###########################
samples_path = "/home/diego/HEP/VBF-Analysis/MuMu/Plots/MasterCR/SR/"
sample1_name = "Zmumu_SherpaRW"
sample2_name = "Zmumu_MGRW"

qcd_1=1.0
qcd_2=1.0

fs_name = "Zmumu"

 ############# Z->ll HISTOS ##########################
histos_no_rebin={
"n_bjets":[],
"lepiso":[],
"n_jets_interval":[],
"delta_R_leplep_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[],
"delta_R_lep1jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[],
"delta_R_lep2jet_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[],    
}
histos_rebin={
"delta_phi":[[2.0],[0.2,0.8],0.2],
"lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[0.1],[0.2,0.199],0.2],
"lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[0.1],[0.2,0.199],0.2],
"ljet0_eta_basic_cuts_ptl":[[-3.0,3.0],[0.5,0.2,0.5],0.2],
"ljet1_eta_basic_cuts_ptl":[[-3.0,3.0],[0.5,0.2,0.5],0.2],
"lep1_pt":[[300],[20,50],20],
"lep2_pt":[[300],[20,50],20],
"ljet0_pt":[[85,460],[85,15,54],15],
"ljet1_pt":[[80,440],[80,15,56],15],
"ljet2_pt_basic_cuts_ptl":[[100],[20,50],20],
"pt_bal":[[0.15,0.3],[0.03,0.05,0.7],0.15],
"Z_centrality":[[0.5],[0.1,0.5],0.1],
"delta_y":[[2.0,6.0],[1.0,0.5,1.0],1.0],
"inv_mass":[[70,110,140],[10,5,10,20],5],
"mass_jj":[[1500],[250,500],250],
"Z_pt_reco_basic_cuts_ptl":[[300],[20,50],20],
"vec_sum_pt_jets_basic_cuts_ptl":[[300],[20,50],20],
"ratio_zpt_sumjetpt_basic_cuts_ptl":[[0.75,1.25],[0.25,0.1,0.25],0.1],
"met_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl":[[50],[10,50],10],
}
total_histos ={**histos_no_rebin,**histos_rebin}

In [17]:
r.TH1.AddDirectory(r.kFALSE)
file1 = r.TFile.Open(samples_path+sample1_name+'.root')
file2 = r.TFile.Open(samples_path+sample2_name+'.root')

myFile =r.TFile.Open(fs_name+"_Average.root", "RECREATE")
r.TH1.AddDirectory(r.kFALSE)    

for histo in total_histos:

    h1 = file1.Get(histo)
    h2 = file2.Get(histo)
    
    
    
    h1.Scale(1/h1.Integral(1,-1))
    h2.Scale(1/h2.Integral(1,-1))
    
    final_hist = h1.Clone()
    final_hist.Add(h2)
    final_hist.Scale(0.5)
    
    if histo in histos_rebin:
        
        print(histo)
    
        rebining=biner(total_histos[histo][0],total_histos[histo][1],h1)
        print(rebining)
        nb=len(rebining)-1
        h1=h1.Rebin(nb,histo,rebining)
        h2=h2.Rebin(nb,histo,rebining)
        
        h1.Scale(qcd_1)
        h2.Scale(qcd_2)

        hist_list=[h1,h2]
        normalization(hist_list,total_histos[histo][2])
    
    myFile.WriteObject(final_hist,histo)

myFile.Close()

os.system("mv "+fs_name+"_Average.root "+samples_path+"/"+fs_name+"_Average.root")

delta_phi
[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  3.2]
lep1_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
[-2.5 -2.3 -2.1 -1.9 -1.7 -1.5 -1.3 -1.1 -0.9 -0.7 -0.5 -0.3 -0.1  0.1
  0.3  0.5  0.7  0.9  1.1  1.3  1.5  1.7  1.9  2.1  2.3  2.5]
lep2_eta_basic_dphi_drap_btag_iso_pt1_pt2_j1pt_j2pt_ptbal_mjj_nji_zcen_mass_ptl
[-2.5 -2.3 -2.1 -1.9 -1.7 -1.5 -1.3 -1.1 -0.9 -0.7 -0.5 -0.3 -0.1  0.1
  0.3  0.5  0.7  0.9  1.1  1.3  1.5  1.7  1.9  2.1  2.3  2.5]
ljet0_eta_basic_cuts_ptl
[-7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.8 -2.6 -2.4 -2.2 -2.
 -1.8 -1.6 -1.4 -1.2 -1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8
  1.   1.2  1.4  1.6  1.8  2.   2.2  2.4  2.6  2.8  3.   3.5  4.   4.5
  5.   5.5  6.   6.5  7. ]
ljet1_eta_basic_cuts_ptl
[-7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.8 -2.6 -2.4 -2.2 -2.
 -1.8 -1.6 -1.4 -1.2 -1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8
  1.   1.2  1.4  1.6  1.8  2.   2.2  2.4  2.6  2.8  3.   3.5  4.   4.5
  5.   5.5  6

0

Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 7 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
